In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matsim
import geojson
import yaml

# Path

In [68]:
root_path = r'../../../../data/intermediate/test/freightEmissions/'
iter_kw = "31"
basic_scenario_outputs = 'scenarioBasic/iter'+iter_kw+'/outputs/'
van_scenario_outputs = 'scenarioVan/iter'+iter_kw+'/outputs/'
cargo_bike_scenario_outputs = 'scenarioCB/iter' + iter_kw +'/outputs/'
scenario_kw = van_scenario_outputs
output_path = r'../../../../data/intermediate/test/freightEmissions/' + scenario_kw+ '/simWrapperData/'

# Utils

In [34]:
def read_matsim_events_as_df(file_path: str, event_types:str):
    events = matsim.event_reader(file_path, types=event_types)
    # Get event keys
    event_keys = set()
    events_list = []
    for event in events:
        events_list.append(event)
        current_keys = list(event.keys())
        event_keys.update(current_keys)
    # Store events in a dict
    events_dict = {}
    for idx, event in enumerate(events_list):
        event_dict = {}
        for key in event_keys:
            if key in event.keys():
                event_dict[key] = event[key]
            else:
                event_dict[key] = None
        events_dict[idx] = event_dict
    # Convert to DataFrame
    events_df = pd.DataFrame.from_dict(events_dict, orient='index')
    # Convert columns which is number-like str into float and ignore the rest
    for column in events_df.columns:
        try:
            events_df[column] = events_df[column].astype(float)
        except:
            pass
    return events_df


# Read events

In [3]:
emission_events = matsim.event_reader(root_path+scenario_kw+'output_events.xml.gz', types='coldEmissionEvent,warmEmissionEvent')
emission_events

<generator object event_reader at 0x0000021C88692880>

# Convert to DataFrame

In [4]:
event_keys = set()
events_list = []
for emission_event in emission_events:
    # type(emission_event)
    # print(emission_event)
    events_list.append(emission_event)
    current_keys = list(emission_event.keys())
    event_keys.update(current_keys)
event_keys

{'BC_exhaust',
 'Benzene',
 'CH4',
 'CO',
 'CO2_TOTAL',
 'CO2_rep',
 'CO2e',
 'FC',
 'FC_MJ',
 'HC',
 'N2O',
 'NH3',
 'NMHC',
 'NO2',
 'NOx',
 'PM',
 'PM2_5',
 'PM_non_exhaust',
 'PN',
 'Pb',
 'SO2',
 'linkId',
 'time',
 'type',
 'vehicleId'}

In [26]:
events_dict = {}
for idx, event in enumerate(events_list):
    event_dict = {}
    for key in event_keys:
        if key in event.keys():
            event_dict[key] = event[key]
        else:
            event_dict[key] = None
    events_dict[idx] = event_dict
    

In [27]:
events_df = pd.DataFrame.from_dict(events_dict, orient='index')
events_df

,Benzene,Pb,NH3,NMHC,linkId,type,FC,PN,PM_non_exhaust,N2O,...,FC_MJ,CO2e,SO2,PM,HC,CO2_rep,CO2_TOTAL,CH4,vehicleId,NO2
0,0.288929879665375,0.0,None,4.0106520652771,27566523_11,coldEmissionEvent,52.3460388183594,0.0,None,None,...,2.21796584129333,193.72981262207,7.79413850978017E-4,0.0,4.24272966384888,154.470077514648,161.47184753418,0.232077300548553,freight_carrier_0_veh_carrier0_Van_0_1,0.0152425747364759
1,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
2,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
3,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
4,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,3.8280981384707365E-6,0.0,0.0014942834489833432,2.2968589500664639E-4,1012971580_0,warmEmissionEvent,3.3350304256272856,5.094064836795652E9,0.0014960097075195629,6.613628880535029E-5,...,0.1413093309859155,None,4.96574164590148E-5,2.8993235275231763E-5,3.828098071486719E-4,9.841479485557981,10.287570505778264,1.531239255388296E-4,freight_carrier_0_veh_carrier0_Van_0_1,9.805452191253604E-5
2943,1.4287446975839973E-6,0.0,5.577050214484201E-4,8.572468435505584E-5,27566523_r_7,warmEmissionEvent,1.2447191436946257,1.9012360345673814E9,5.583493055396498E-4,2.4683764242856134E-5,...,0.05274027730279731,None,1.8533425187965582E-5,1.0821021213902546E-5,1.428744672583839E-4,3.673093301884308,3.8395859456790085,5.714978790335994E-5,freight_carrier_0_veh_carrier0_Van_0_1,3.659646988899699E-5
2944,4.864895997052162E-7,0.0,1.898993522753423E-4,2.9189376833571974E-5,27566523_r_8,warmEmissionEvent,0.4238286371143767,6.473735713356447E8,1.9011873192390289E-4,8.404856799140648E-6,...,0.01795813936301347,None,6.310657611612847E-6,3.6845730994872915E-6,4.8648959119262675E-5,1.2506934885814707,1.3073844703716482,1.9459583988208663E-5,freight_carrier_0_veh_carrier0_Van_0_1,1.2461149999036437E-5
2945,9.386542757441719E-7,0.0,2.7730770612650424E-4,5.631926058467022E-5,27566523_r_9,warmEmissionEvent,0.6798415795019153,1.1991723568915517E9,3.7373007942753705E-4,1.2273509790133616E-5,...,0.028805723531020424,None,1.0122599432885608E-5,7.126424289808496E-6,9.386543099289556E-5,2.006172682227888,2.097107758784033,3.754617289439142E-5,freight_carrier_0_veh_carrier0_Van_0_1,1.9474489851411906E-5


In [28]:
# Convert columns which is number-like str into float and ignore the rest
for column in events_df.columns:
    try:
        events_df[column] = events_df[column].astype(float)
    except:
        pass

# Analysis

## Aggregate by link

In [29]:
link_co2e = events_df.pivot_table(index='linkId',
                                       columns='type', 
                                       values='CO2e', 
                                       aggfunc='sum')
link_co2e

type,coldEmissionEvent,warmEmissionEvent
linkId,,
100344071_8,14.704374,0.0
100490507_0,NaN,0.0
100490507_1,NaN,0.0
100490507_2,NaN,0.0
100490507_3-100490507_4,NaN,0.0
...,...,...
splitLink_123929657_r_1_2,NaN,0.0
splitLink_15216997_8_1,NaN,0.0
splitLink_15216997_8_2,NaN,0.0


In [30]:
link_co2e = link_co2e.fillna(0)
link_co2e['sum'] = link_co2e[['coldEmissionEvent', 'warmEmissionEvent']].sum(axis=1)
link_co2e 

type,coldEmissionEvent,warmEmissionEvent,sum
linkId,,,
100344071_8,14.704374,0.0,14.704374
100490507_0,0.000000,0.0,0.000000
100490507_1,0.000000,0.0,0.000000
100490507_2,0.000000,0.0,0.000000
100490507_3-100490507_4,0.000000,0.0,0.000000
...,...,...,...
splitLink_123929657_r_1_2,0.000000,0.0,0.000000
splitLink_15216997_8_1,0.000000,0.0,0.000000
splitLink_15216997_8_2,0.000000,0.0,0.000000


In [31]:
link_co2e['sum'].sum()

6108.871200561526

# Read move events

In [45]:
move_events = matsim.event_reader(root_path+scenario_kw+'output_events.xml.gz', types='entered link,left link')
move_events

<generator object event_reader at 0x0000021C8C88A8F0>

In [46]:
event_keys = set()
events_list = []
for move_event in move_events:
    # type(emission_event)
    # print(emission_event)
    events_list.append(move_event)
    current_keys = list(move_event.keys())
    event_keys.update(current_keys)
event_keys

{'link', 'time', 'type', 'vehicle'}

In [47]:
events_dict = {}
for idx, event in enumerate(events_list):
    event_dict = {}
    for key in event_keys:
        if key in event.keys():
            event_dict[key] = event[key]
        else:
            event_dict[key] = None
    events_dict[idx] = event_dict

In [48]:
move_events_df = pd.DataFrame.from_dict(events_dict, orient='index')
move_events_df

,time,link,vehicle,type
0,11.0,333784188_r_3,freight_carrier_0_veh_carrier0_CB_1_1,left link
1,11.0,326736776_r_5,freight_carrier_0_veh_carrier0_CB_1_1,entered link
2,11.0,27566523_11,freight_carrier_0_veh_carrier0_CB_0_2,left link
3,11.0,27566523_r_9,freight_carrier_0_veh_carrier0_CB_0_2,entered link
4,13.0,326736776_r_5,freight_carrier_0_veh_carrier0_CB_1_1,left link
...,...,...,...,...
30113,42735.0,27566523_r_8,freight_carrier_0_veh_carrier0_CB_0_2,entered link
30114,42737.0,27566523_r_8,freight_carrier_0_veh_carrier0_CB_0_2,left link
30115,42737.0,27566523_r_9,freight_carrier_0_veh_carrier0_CB_0_2,entered link
30116,42739.0,27566523_r_9,freight_carrier_0_veh_carrier0_CB_0_2,left link


In [9]:
move_events_df['vehicle'].unique()

array(['freight_carrier_0_veh_carrier0_Van_0_1'], dtype=object)

In [62]:
move_events_df.query('vehicle == "freight_carrier_0_veh_carrier0_CB_0_2"')

,time,vehicle,type,link
0,8.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,333784188_r_3
1,8.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,326736776_r_5
4,10.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,326736776_r_5
5,10.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,326736776_r_6
8,13.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,326736776_r_6
...,...,...,...,...
30919,42423.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,326736776_4
30920,42425.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,326736776_4
30921,42425.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,333784188_0
30922,42429.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,333784188_0


In [49]:
def derive_vehicle_move_chain(move_events_df):
    vehicles_move_chain = {}
    for vehicle in move_events_df['vehicle'].unique():
        sub_df = move_events_df.query('vehicle == @vehicle')
        visited_links = []
        for idx, row in sub_df.iterrows():
            if row['link'] not in visited_links:
                visited_links.append(row['link'])
        vehicles_move_chain[vehicle] = visited_links
    return vehicles_move_chain

In [50]:
v_move_chains = derive_vehicle_move_chain(move_events_df)

In [12]:
def cal_vehicle_travel_distance(vehicle_move_chain: list, network: gpd.GeoDataFrame):
    total_distance = 0
    for idx, link_id in enumerate(vehicle_move_chain):
        link = network.query('link_id == @link_id')
        total_distance += link['length'].values[0]
    return total_distance


In [18]:
def cal_vkt(v_move_chain: list, network: gpd.GeoDataFrame):
    # Create a dataframe of chains
    vkt_dict = {}
    for idx, link in enumerate(v_move_chain):
        vkt_dict[idx] = {'link': link}
    vkt_df = pd.DataFrame.from_dict(vkt_dict, orient='index')
    # Join with network
    vkt_df = vkt_df.merge(network, left_on='link', right_on='link_id', how='left')
    vkt = vkt_df['length'].sum()
    return vkt

# Read network

In [13]:
network = matsim.read_network(
    root_path + 'diffusedGemeenteLeuvenWithHbefaType.xml.gz'
)
network

In [14]:
network_gdf = network.as_geo()
network_gdf

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,7.221720,1.543210,1030.864198,2.0,1,"bike,car",100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.462 171579.032, 175087.718 ..."
1,7.924752,1.543210,1030.864198,2.0,1,"bike,car",100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ..."
2,8.518773,1.543210,1030.864198,2.0,1,"bike,car",100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.774, 175106.042 ..."
3,50.128561,10.030864,2200.617284,3.5,1,"bike,car",100010104_0,1156293852,1156293879,"LINESTRING (174065.630 177001.873, 174056.230 ..."
4,14.524426,10.030864,2200.617284,3.5,1,"bike,car",100010104_1,1156293879,1156293875,"LINESTRING (174056.230 177051.111, 174070.620 ..."
...,...,...,...,...,...,...,...,...,...,...
103748,183.145612,10.030864,2200.617284,3.5,1,car,splitLink_9904390_5_2,splitNode_9904390_5_1,19792038,"LINESTRING (175632.401 172452.198, 175505.340 ..."
103749,102.305105,10.030864,2200.617284,3.5,1,car,splitLink_9904410_0_1,80702464,splitNode_9904410_0_1,"LINESTRING (176002.946 172889.690, 176056.498 ..."
103750,102.305105,10.030864,2200.617284,3.5,1,car,splitLink_9904410_0_2,splitNode_9904410_0_1,100399122,"LINESTRING (176056.498 172976.855, 176110.050 ..."
103751,113.694402,1.543210,1030.864198,2.0,1,"bike,car",splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ..."


In [15]:
traveled_link = network_gdf[network_gdf['link_id'].isin(move_events_df['link'])]
traveled_link

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
109,5.188792,10.030864,2200.617284,3.5,1,"bike,car",100344071_0,1026364650,5351231265,"LINESTRING (172557.615 173619.591, 172555.721 ..."
110,59.966019,10.030864,2200.617284,3.5,1,"bike,car",100344071_1-100344071_2,5351231265,1026364266,"LINESTRING (172555.721 173624.421, 172516.672 ..."
111,7.259080,10.030864,2200.617284,3.5,1,"bike,car",100344071_3,1026364266,1026364514,"LINESTRING (172516.672 173669.706, 172510.112 ..."
112,8.604302,10.030864,2200.617284,3.5,1,"bike,car",100344071_4,1026364514,1026364360,"LINESTRING (172510.112 173672.815, 172501.637 ..."
113,6.361050,10.030864,2200.617284,3.5,1,"bike,car",100344071_5,1026364360,1026364261,"LINESTRING (172501.637 173671.332, 172496.805 ..."
...,...,...,...,...,...,...,...,...,...,...
102826,134.172218,10.030864,2200.617284,3.5,1,"bike,car",splitLink_15216997_8_2,splitNode_15216997_8_1,150492900,"LINESTRING (174117.515 175123.802, 174004.529 ..."
102827,134.172218,10.030864,2200.617284,3.5,1,"bike,car",splitLink_15216997_r_12_1,150492900,splitNode_15216997_r_12_1,"LINESTRING (174004.529 175196.155, 174117.515 ..."
102828,134.172218,10.030864,2200.617284,3.5,1,"bike,car",splitLink_15216997_r_12_2,splitNode_15216997_r_12_1,150492899,"LINESTRING (174117.515 175123.802, 174230.501 ..."
103344,105.524047,10.030864,1200.617284,3.5,1,"bike,car",splitLink_3991677_1_1,5143418387,splitNode_3991677_1_1,"LINESTRING (174064.906 174452.996, 173987.433 ..."


In [51]:
for vehicle, move_chain in v_move_chains.items():
    print(vehicle, cal_vkt(move_chain, network_gdf))

freight_carrier_0_veh_carrier0_CB_1_1 40452.71405930634
freight_carrier_0_veh_carrier0_CB_0_2 70982.15357802532


# Read tour events

In [69]:
tour_events_df = read_matsim_events_as_df(root_path+scenario_kw+'output_events.xml.gz', 'Freight tour starts,Freight tour ends')
tour_events_df

,time,link,tourId,carrierId,vehicle,type
0,0.0,2.756652e+09,1.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight tour starts
1,8339.0,2.756652e+09,1.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight tour ends


In [56]:
def calculate_tour_durations(df):

    tour_durations = {}

    grouped = df.groupby(['vehicle', 'tourId'])


    for (vehicle, tourId), group in grouped:
   
        if 'Freight tour starts' in group['type'].values and 'Freight tour ends' in group['type'].values:
            start_time = group[group['type'] == 'Freight tour starts']['time'].values[0]
            end_time = group[group['type'] == 'Freight tour ends']['time'].values[0]
            duration = end_time - start_time
            
            tour_durations[(vehicle, tourId)] = duration

    # Calculate the total duration of each vehicle
    vehicle_durations = {}
    for (vehicle, tourId), duration in tour_durations.items():
        if vehicle in vehicle_durations:
            vehicle_durations[vehicle] += duration
        else:
            vehicle_durations[vehicle] = duration

    return tour_durations, vehicle_durations


In [70]:
calculate_tour_durations(tour_events_df)

({('freight_carrier_0_veh_carrier0_Van_0_1', 1.0): 8339.0},
 {'freight_carrier_0_veh_carrier0_Van_0_1': 8339.0})

# Read shipment pickup events

In [71]:
shipment_events = read_matsim_events_as_df(root_path+scenario_kw+'output_events.xml.gz', 'Freight shipment pickup starts')
shipment_events

,time,link,capacityDemand,shipmentId,pickupDuration,carrierId,vehicle,type
0,0.0,27566523_11,6.0,shipment_184,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
1,1.0,27566523_11,23.0,shipment_124,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
2,2.0,27566523_11,17.0,shipment_25,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
3,3.0,27566523_11,12.0,shipment_102,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
4,4.0,27566523_11,19.0,shipment_120,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
...,...,...,...,...,...,...,...,...
195,980.0,333784188_r_3,12.0,shipment_40,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
196,981.0,333784188_r_3,16.0,shipment_158,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
197,982.0,333784188_r_3,23.0,shipment_103,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts
198,983.0,333784188_r_3,13.0,shipment_86,0.0,carrier_0,freight_carrier_0_veh_carrier0_Van_0_1,Freight shipment pickup starts


In [42]:
def aggregate_vehicle_shipment_demand(shipment_events_df):
    vehicle_capacity_demand = {}
    for vehicle in shipment_events_df['vehicle'].unique():
        sub_df = shipment_events_df.query('vehicle == @vehicle')
        capacity_demand = sub_df['capacityDemand'].sum()
        vehicle_capacity_demand[vehicle] = capacity_demand
    return vehicle_capacity_demand

In [72]:
aggregate_vehicle_shipment_demand(shipment_events)

{'freight_carrier_0_veh_carrier0_Van_0_1': 3000.0}

# Output 

In [41]:
link_co2e.to_csv(output_path+van_scenario_outputs+'link_co2e.csv', index=True, sep=';', encoding='utf-8-sig')